## Hugging Face : martin-ha/toxic-comment-model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [ ]:
# Load model & tokenizer
model_name = "martin-ha/toxic-comment-model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/403 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [ ]:
# Sample text
# text = "I'm not wearing anything under this dress 😉"
text = "I want to beat the hell out of you"
# text = "I will kill you"

# Encode
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
probs = F.softmax(outputs.logits, dim=1)

# Labels (from model card)
labels = ["toxic", "obscene", "threat", "insult", "identity_attack", "sexually_explicit"]

# Print results
for label, score in zip(labels, probs[0].detach().numpy()):
    print(f"{label}: {score:.4f}")


toxic: 0.0458
obscene: 0.9542


## Hugging face : unitary/unbiased-toxic-roberta

In [ ]:
# Import the pipeline function from the transformers library
from transformers import pipeline

In [ ]:
# 1. Load the text classification pipeline with the specific model
# Setting return_all_scores=True shows the model's confidence for every category
text_classifier = pipeline(
    "text-classification",
    model="unitary/unbiased-toxic-roberta",
    return_all_scores=True
)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/997 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
# 2. Provide the text you want to analyze
# This example has a suggestive undertone
text_to_check = "I can't stop thinking about what you were wearing last night."

In [ ]:
# 3. Classify the text
# The model requires the input to be a list of strings
results = text_classifier([text_to_check])

In [ ]:
# 4. Print the results in a readable format
print(f"Text Analysis for: '{text_to_check}'")
# The output is a list containing one dictionary of scores
for label in results[0]:
    label_name = label['label']
    score = label['score']
    print(f"- Label: {label_name}, Score: {score:.4f}")

Text Analysis for: 'I can't stop thinking about what you were wearing last night.'
- Label: toxicity, Score: 0.0006
- Label: severe_toxicity, Score: 0.0000
- Label: obscene, Score: 0.0000
- Label: identity_attack, Score: 0.0000
- Label: insult, Score: 0.0002
- Label: threat, Score: 0.0000
- Label: sexual_explicit, Score: 0.0000
- Label: male, Score: 0.0002
- Label: female, Score: 0.0001
- Label: homosexual_gay_or_lesbian, Score: 0.0000
- Label: christian, Score: 0.0001
- Label: jewish, Score: 0.0000
- Label: muslim, Score: 0.0000
- Label: black, Score: 0.0000
- Label: white, Score: 0.0001
- Label: psychiatric_or_mental_illness, Score: 0.0001


## Hugging face : facebook/bart-large-mnli

In [ ]:
# Ensure you have the necessary libraries installed:
# pip install transformers torch

from transformers import pipeline

def check_for_suggestive_content(text_to_analyze: str) -> dict:
    """
    Analyzes a given text to determine if it's suggestive using a
    zero-shot classification model.

    Args:
        text_to_analyze: The input string to be classified.

    Returns:
        A dictionary containing the text, the predicted label, and the confidence score.
    """
    # 1. Initialize the zero-shot classification pipeline
    # This model is trained to understand if a premise (your text) entails a hypothesis (a label).
    classifier = pipeline("zero-shot-classification",
                          model="facebook/bart-large-mnli")

    # 2. Define the text and candidate labels
    # The 'candidate_labels' act as the core of the "prompt."
    # The model will determine which label is the most likely description of the input text.
    candidate_labels = ["suggestive content", "safe content"]

    # 3. Classify the text
    # The model calculates a probability score for each label.
    result = classifier(text_to_analyze, candidate_labels)

    # 4. Format and return the result
    return {
        "text": result["sequence"],
        "prediction": result["labels"][0],
        "confidence": round(result["scores"][0], 4)
    }

In [ ]:
# Example 1: A potentially suggestive text
text1 = "Come see my private photos and videos, available for a limited time tonight."
result1 = check_for_suggestive_content(text1)
print(f"Analysis for: '{result1['text']}'")
print(f"  -> Prediction: {result1['prediction']} (Confidence: {result1['confidence']})\n")

Device set to use cpu


Analysis for: 'Come see my private photos and videos, available for a limited time tonight.'
  -> Prediction: suggestive content (Confidence: 0.5772)



In [ ]:
# Example 2: A clearly safe text
text2 = "Our community bake sale is this Saturday. We will have cookies, pies, and bread."
result2 = check_for_suggestive_content(text2)
print(f"Analysis for: '{result2['text']}'")
print(f"  -> Prediction: {result2['prediction']} (Confidence: {result2['confidence']})\n")

Device set to use cpu


Analysis for: 'Our community bake sale is this Saturday. We will have cookies, pies, and bread.'
  -> Prediction: suggestive content (Confidence: 0.5289)



In [ ]:
# Example 3: A more subtle case
text3 = "I'm feeling lonely tonight, looking for someone to chat with."
result3 = check_for_suggestive_content(text3)
print(f"Analysis for: '{result3['text']}'")
print(f"  -> Prediction: {result3['prediction']} (Confidence: {result3['confidence']})")

Device set to use cpu


Analysis for: 'I'm feeling lonely tonight, looking for someone to chat with.'
  -> Prediction: suggestive content (Confidence: 0.8927)


## Hugging face : meta-llama/Llama-3.1-8B-Instruct

In [ ]:
import torch
from transformers import pipeline
import re

In [ ]:
hf_token = '.................'

generator = pipeline(
        "text-generation",
        model="meta-llama/Llama-3.1-8B-Instruct",
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
        token=hf_token # Pass the token here
    )

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
text_to_analyze = "Come see my private photos and videos, available for a limited time tonight."
# text_to_analyze = 'Our community bake sale is this Saturday. We will have cookies, pies, and bread.'

In [ ]:
messages = [
  {
      "role": "system",
      "content": "You are an expert content moderator. Your task is to classify text as either 'suggestive' or 'safe'. Do not provide any explanations or extra text. Your response must be only one word: 'suggestive' or 'safe'."
  },
  {
      "role": "user",
      "content": f"Please classify the following text: \"{text_to_analyze}\""
  },]

In [ ]:
outputs = generator(messages, max_new_tokens=5, return_full_text=False)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(outputs)

[{'generated_text': 'safe'}]


In [ ]:
prediction = outputs[0]['generated_text'].strip().lower()

In [ ]:
if 'suggestive' in prediction:
    final_label = 'suggestive'
else:
    final_label = 'safe'
print(final_label)

safe


## Hugging face : google/gemma-2-9b-it

In [ ]:
import torch
from transformers import pipeline
import re

In [ ]:
hf_token = '..............'
generator = pipeline(
  "text-generation",
  model="google/gemma-2-9b-it", # <-- The only major change is here!
  model_kwargs={"torch_dtype": torch.bfloat16},
  device_map="auto",
  token=hf_token)

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
text_to_analyze = "Come see my private photos and videos, available for a limited time tonight."
# text_to_analyze = 'Our community bake sale is this Saturday. We will have cookies, pies, and bread.'

In [ ]:
messages = [
  {
    "role": "user",
    "content": """You are a content moderator. Your task is to classify the following text as either 'suggestive' or 'safe'.Your response must be only one word: 'suggestive' or 'safe'. Do not provide any explanations or extra text.
Classify this text: \"{}\"""".format(text_to_analyze)
  },]

In [ ]:
outputs = generator(messages, max_new_tokens=5)

In [ ]:
outputs

[{'generated_text': [{'role': 'user',
    'content': 'You are a content moderator. Your task is to classify the following text as either \'suggestive\' or \'safe\'.Your response must be only one word: \'suggestive\' or \'safe\'. Do not provide any explanations or extra text.\nClassify this text: "Come see my private photos and videos, available for a limited time tonight."'},
   {'role': 'assistant', 'content': 'suggestive \n'}]}]

In [ ]:
generated_text = outputs[0]['generated_text'][-1]['content']
prediction = generated_text.strip().lower()

if 'suggestive' in prediction:
    final_label = 'suggestive'
else:
    final_label = 'safe'
print(final_label)

suggestive


## Hugging face : meta-llama/Llama-3.1-8B-Instruct for multi-label classification

### NOTE : below code only classify the text into only one these labels
<br>(Explicit Nudity Detection
Suggestive Content Detection
Violence Detection
Visually Disturbing Content
Rude Gestures Detection
Alcohol Content Detection
Drugs Content Detection
Tobacco Content Detection
Hate Symbol Detection)

### but for a text multiple labels can be possible

In [ ]:
import torch
from transformers import pipeline
import re

In [ ]:
hf_token = '....................'

generator = pipeline(
        "text-generation",
        model="meta-llama/Llama-3.1-8B-Instruct",
        model_kwargs={"torch_dtype": torch.bfloat16},
        device_map="auto",
        token=hf_token # Pass the token here
    )

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
valid_labels = [
  "explicit_nudity", "suggestive", "violence", "disturbing_content",
  "rude_gestures", "alcohol", "drugs", "tobacco", "hate_speech", "safe"]

In [ ]:
labels_for_prompt = ", ".join(f"'{label}'" for label in valid_labels)

In [ ]:
# text_to_analyze = "He drew his sword and charged, intending to cut them down where they stood."  #Violence
# text_to_analyze = 'Our community bake sale is this Saturday. We will have cookies, pies, and bread.'
text_to_analyze="We spent the whole night drinking beer and shots of whiskey at the downtown bar." #Alcohol
# text_to_analyze="The dealer was caught selling pills and white powder on the street corner." #Drugs
# text_to_analyze="He lit up a cigarette and took a long drag, enjoying the smoke."  #Tobacco
# text_to_analyze="All people of that background are inferior and shouldn't be allowed here." #Hate Speech
# text_to_analyze="The community is organizing a bake sale to raise funds for the local park." #Safe Content
# text_to_analyze="The driver got angry and showed him the middle finger before speeding off."  #Rude Gestures

In [ ]:
messages = [
  {
    "role": "system",
    "content": f"You are an expert content moderator. Your task is to analyze the user's text and classify it into ONE of the following categories: {labels_for_prompt}. Your response must be only the single, most appropriate category name from this list and nothing else."
  },
  {
    "role": "user",
    "content": f"Please classify the following text: \"{text_to_analyze}\""
  },]

In [ ]:
outputs = generator(messages, max_new_tokens=8, return_full_text=False)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(outputs)

[{'generated_text': 'alcohol'}]


In [ ]:
prediction = outputs[0]['generated_text'].strip().lower()

In [ ]:
final_label = 'safe'
for label in valid_labels:
  # Check if the model's output contains one of our valid labels.
  if label in prediction:
    final_label = label
    break # Stop after finding the first valid match
final_label

'violence'

## Hugging face : meta-llama/Llama-3.1-8B-Instruct for multi-label classification : <font color="blue">improved version</font>

In [ ]:
import torch
from transformers import pipeline
import time

In [ ]:
hf_token = '......................'
generator = pipeline(
    "text-generation",
    model="meta-llama/Llama-3.1-8B-Instruct",
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
    token=hf_token
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
valid_labels = ["explicit_nudity", "suggestive", "violence", "disturbing_content","rude_gestures", "alcohol", "drugs", "tobacco", "hate_speech", "safe"]

In [ ]:
labels_for_prompt = ", ".join(f"'{label}'" for label in valid_labels)

In [ ]:
text_to_analyze='He was drunk. He drew his sword and charged, intending to cut them down where they stood.'

In [ ]:
messages = [
  {
    "role": "system",
    "content": f"You are an expert content moderator. Your task is to identify ALL applicable categories for the user's text from the following list: {labels_for_prompt}. Your response MUST be a comma-separated list of the category names. If none of the categories apply, respond with only the word 'safe'."
  },
  {
    "role": "user",
    "content": f"Please classify the following text: \"{text_to_analyze}\""
  },]

In [ ]:
start_time = time.time()
outputs = generator(messages, max_new_tokens=40, return_full_text=False)
end_time = time.time()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [ ]:
print(outputs)
print(f"inferencing finished in {end_time - start_time:.2f} seconds.")

[{'generated_text': 'alcohol, violence'}]
inferencing finished in 129.44 seconds.


In [ ]:
raw_prediction = outputs[0]['generated_text'].strip().lower()

In [ ]:
potential_labels = [label.strip() for label in raw_prediction.split(',')]
final_labels = [label for label in potential_labels if label in valid_labels]
if not final_labels or final_labels == ['safe']:
    final_labels = ['safe']

In [ ]:
print(final_labels)

['alcohol', 'violence']
